<a href="https://colab.research.google.com/github/urmzd/rabbit-classifier/blob/main/src/rc_e1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!curl https://raw.githubusercontent.com/urmzd/rabbit-classifier/main/resources/data.csv --output data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9419  100  9419    0     0   141k      0 --:--:-- --:--:-- --:--:--  141k


In [21]:
import pandas as pd

DATA_PATH="/content/data.csv"
pd.read_csv(DATA_PATH, usecols=range(2))

,label,link
0,rabbit,https://upload.wikimedia.org/wikipedia/commons...
1,rabbit,https://upload.wikimedia.org/wikipedia/commons...
2,rabbit,https://www.massaudubon.org/var/ezdemo_site/st...
3,rabbit,https://www.welcomewildlife.com/wp-content/upl...
4,rabbit,https://i.natgeofe.com/k/58df97a7-5c47-44b8-97...
...,...,...
81,hare,https://videohive.img.customer.envatouserconte...
82,hare,https://i.ytimg.com/vi/8hEKqUG-WVc/maxresdefau...
83,hare,https://www.furfeatherandfin.com/media/wysiwyg...
84,hare,https://www.wildlifetrusts.org/sites/default/f...


In [ ]:
!pip install pytorch_lightning
!pip install wandb

In [ ]:
!wandb login

wandb: Currently logged in as: csci-4155-rabbit-classifier (use `wandb login --relogin` to force relogin)


In [ ]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer

wandb_logger = WandbLogger(project="experiment-one")